In [1]:
# GENERAL IMPORTS

import torch
from mapping.datasets import RLDatasetFormatter
from mapping.models import Discriminator, TransitionModel, LitAutoEncoder
from mapping.config import LOGGING_DIR, TRAIN_WEIGHTS, ITERATIONS

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# READING OF DATASETS

dataset_t_path = 'data/UntrainedMCDataset500.csv'
dataset_s_path = 'data/UntrainedPendDataset5000.csv'
extended_dataset_t_path = 'data/UntrainedMCDataset5000.csv'
test_dataset_s_path = 'data/UntrainedPendDataset500.csv'
super_extended_dataset_t_path = 'data/UntrainedMCDataset50000.csv'


data_formatter_t = RLDatasetFormatter().from_csv(dataset_t_path)
data_formatter_s = RLDatasetFormatter().from_csv(dataset_s_path)
test_data_formatter_s = RLDatasetFormatter().from_csv(test_dataset_s_path)
ext_data_formatter_t = RLDatasetFormatter().from_csv(extended_dataset_t_path)
super_ext_data_formatter_t = RLDatasetFormatter().from_csv(super_extended_dataset_t_path)

In [3]:
dataset_t = data_formatter_t.as_transitions()

T = TransitionModel(data_formatter_t.state_size, data_formatter_t.action_size)
T.train_model(dataset_t, model_dir='/T')

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name           | Type       | Params
----------------------------------------------
0 | loss_criterion | L1Loss     | 0     
1 | statefc        | Sequential | 8.5 K 
2 | actionfc       | Sequential | 8.4 K 
3 | predfc         | Sequential | 16.6 K
----------------------------------------------
33.5 K    Trainable params
0         Non-trainable params
33.5 K    Total params
0.134     Total estimated model params size (MB)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pytorch_lightning/loggers/tensorboard.py:262: UserWarning: Could not log computational graph to TensorBoard: The `model.example_input_array` attribute is not set or `input_array` was not given.
  "Could not log computational graph to TensorBoard: The `model.example_input_array` attribute"


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:229: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:229: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:156

Epoch 99: 100%|██████████| 26/26 [00:00<00:00, 145.00it/s, loss=0.00196, v_num=1]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 26/26 [00:00<00:00, 139.27it/s, loss=0.00196, v_num=1]


In [4]:
stringa = ''

test_dataset_s = test_data_formatter_s.transition_identity()
dataset_t = data_formatter_t.transition_identity()
dataset_s = data_formatter_s.transition_identity()

D = Discriminator(data_formatter_t.state_size, data_formatter_t.action_size)
disc_dataset = data_formatter_t.transition_as_valid()
D.train_model(disc_dataset, model_dir='/Discriminator', batch_size=100, epochs=20)

source_triplet_size = data_formatter_s.state_size*2+data_formatter_s.action_size
AE = LitAutoEncoder(T, D, dim_s=source_triplet_size, lambdas=TRAIN_WEIGHTS)

for i in range(ITERATIONS):
#while stringa != 'stop':

#T = TransitionModel.load_from_checkpoint(T_model_checkpoint, s_dim=data_formatter_t.state_size, a_dim=data_formatter_t.action_size)
#D = Discriminator.load_from_checkpoint(D_model_checkpoint, s_dim=data_formatter_t.state_size, a_dim=data_formatter_t.action_size)

    AE._D = D.as_dict()['func']
    AE.train_model(dataset_s, batch_size=200, model_dir="/AE")

    M =  AE.as_dict()['M']

    test_dataset_s.shuffle()
    #dataset_s.shuffle()
    sas, _ = test_dataset_s[:len(dataset_t)]
    
    with torch.no_grad():
        synthetic_data = M(sas)

    r_synth = torch.zeros((synthetic_data.shape[0],1))
    s_synth,a_synth,s1_synth = torch.split(synthetic_data, [data_formatter_t.state_size, data_formatter_t.action_size, data_formatter_t.state_size],1)
    data_formatter_synth = RLDatasetFormatter([s_synth, a_synth, r_synth, s1_synth])


    #disc_dataset = data_formatter_t.transition_as_valid()
    #fake_samples = data_formatter_synth.transition_as_fake()
    #disc_dataset = disc_dataset.merge(fake_samples)
    disc_dataset = data_formatter_synth.transition_as_fake()

    #D = Discriminator(data_formatter_t.state_size, data_formatter_t.action_size)
    D.train_model(disc_dataset, model_dir='/Discriminator', batch_size=100, epochs=20)

    #stringa = input()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name           | Type       | Params
----------------------------------------------
0 | sfc            | Sequential | 8.5 K 
1 | actionfc       | Sequential | 8.4 K 
2 | s1fc           | Sequential | 8.5 K 
3 | model          | Sequential | 24.7 K
4 | loss_criterion | BCELoss    | 0     
----------------------------------------------
50.2 K    Trainable params
0         Non-trainable params
50.2 K    Total params
0.201     Total estimated model params size (MB)


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1562: PossibleUserWarning: The number of training batches (5) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  category=PossibleUserWarning,


Epoch 19: 100%|██████████| 6/6 [00:00<00:00, 50.13it/s, loss=0.233, v_num=8] 

`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|██████████| 6/6 [00:00<00:00, 44.96it/s, loss=0.233, v_num=8]


...

Epoch 99: 100%|██████████| 26/26 [00:00<00:00, 92.28it/s, loss=96.7, v_num=13]


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name           | Type       | Params
----------------------------------------------
0 | sfc            | Sequential | 8.5 K 
1 | actionfc       | Sequential | 8.4 K 
2 | s1fc           | Sequential | 8.5 K 
3 | model          | Sequential | 24.7 K
4 | loss_criterion | BCELoss    | 0     
----------------------------------------------
50.2 K    Trainable params
0         Non-trainable params
50.2 K    Total params
0.201     Total estimated model params size (MB)


Epoch 19: 100%|██████████| 10/10 [00:00<00:00, 38.45it/s, loss=0.000288, v_num=15]

`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|██████████| 10/10 [00:00<00:00, 34.83it/s, loss=0.000288, v_num=15]


In [ ]:
M = lambda sas: AE.encoder(sas)

dataset_s = data_formatter_s.transition_identity()

sas, _ = dataset_s[:]

with torch.no_grad():
    synthetic_data = M(sas)
r_synth = torch.zeros((synthetic_data.shape[0],1))
s_synth,a_synth,s1_synth = torch.split(synthetic_data, [data_formatter_t.state_size, data_formatter_t.action_size, data_formatter_t.state_size],1)
data_formatter_synth = RLDatasetFormatter([s_synth, a_synth, r_synth, s1_synth])

In [ ]:
dataset_synth = data_formatter_synth.as_transitions()
dataset_hybrid = data_formatter_t.as_transitions().merge(dataset_synth)
T_hat = TransitionModel(data_formatter_t.state_size, data_formatter_t.action_size)
T_hat.train_model(dataset_hybrid, model_dir='/T_hat')

In [ ]:
ext_dataset_t = ext_data_formatter_t.as_transitions()

sa, s1 = ext_dataset_t[:]

import matplotlib.pyplot as plt

T.eval()
T_hat.eval()
with torch.no_grad():
    error = T(sa)-s1
    plt.plot(sorted(torch.norm(error,dim=1,).detach().numpy(), reverse=True))
    error = T_hat(sa)-s1
    plt.plot(sorted(torch.norm(error,dim=1).detach().numpy(),reverse=True))

plt.legend(['T', 'T_hat'])
plt.grid()

In [ ]:
ext_dataset_t = ext_data_formatter_t.as_transitions()
T_ext = TransitionModel(data_formatter_t.state_size, data_formatter_t.action_size)
T_ext.train_model(ext_dataset_t, model_dir='/T_ext')

In [ ]:
dataset_synth = data_formatter_synth.as_transitions()
#T_synth = TransitionModel(data_formatter_t.state_size, data_formatter_t.action_size)
#T_synth.train_model(dataset_synth, model_dir='/T_synth')

In [ ]:
super_ext_dataset_t = super_ext_data_formatter_t.as_transitions()

sa, s1 = super_ext_dataset_t[:]

import matplotlib.pyplot as plt

models = [T,T_ext,T_hat#,T_synth
]
model_names = ['Trained on 500 target',
               'Trained on 5000 target',
               'Trained on 500 target + 5000 synthetic',
               #'Trained on 5000 synthetic'
               ]

for m in models:
    m = m.as_dict()['func']
    with torch.no_grad():
        error = torch.norm(m(sa) - s1, dim=1).detach().numpy()
        plt.plot(sorted(error, reverse=True ))


plt.legend(model_names)
plt.title('Transition model trained using different datasets')
plt.grid()

In [ ]:

def plot_state_distribution(dataset, name: str, bins: int = 100):
    """
    Plot the distribution of states in a dataset.
    """
    sa,s1 = dataset[:]
    state_size = s1.shape[1]

    # components = [s1[:,i].detach().numpy() for i in range(state_size)]

    components = [[] for _ in range(state_size)]

    for sa, s1 in dataset:
        for i, c in enumerate(s1):
            components[i].append(c.detach().numpy())

    fig, axs = plt.subplots(state_size)

    for i, c in enumerate(components):
        axs[i].hist(c, bins=bins)
        axs[i].set_ylabel(f"Component {i}")
        axs[i].grid()

    fig.suptitle(
        f"{name}\nState Marginal Distributions"
    )  
    plt.show()



In [ ]:
plot_state_distribution(data_formatter_t.as_transitions(), f"MC Dataset ({len(dataset_t)} samples)")
plot_state_distribution(ext_dataset_t, f"MC Dataset ({len(ext_dataset_t)} samples)")
plot_state_distribution(dataset_synth, f"Mapped Dataset PEND2MC ({len(dataset_synth)} samples)")
plot_state_distribution(dataset_hybrid, f"Hybrid Dataset MC500+Mapped5000 ({len(dataset_t) + len(dataset_synth)} samples)")